# compute_ukca_n10.py

This code calculates N10 (concentration of particles greater than 10 nm) from time series of UKCA aerosol mass and number fields. UKCA lognormal modes are evaluated to a size distribution (dN/dlogD) and then the size distribution is integrated over sizes about 10 nm.

In [ ]:
import numpy as np
import capricorn_constants as const
import capricorn_functions as cfn
from importlib import reload
import xarray as xr
reload(const)
reload(cfn)

In [ ]:
def lognormal_pdf(mean, sigma, bins):
    """
    returns pdf of specified lognormal dist at points given by bins
    formula adapted from exmaple on numpy.random.lognormal webpage
    returns pdf as a function of log(bins)
    """
    if mean.shape != sigma.shape:
        print(f"\n[lognormal_pdf] [WARNING] shape of mean ({mean.shape}) doesn't match shape of sigma ({sigma.shape})")
        print(f"[lognormal_pdf] [WARNING] code will fail or produce unexpected behaviour")
    # first enusre bins can be broadcast to mean and sigma
    # !!! this only works for a specific shape of mean and sigma
    bins = np.expand_dims(bins, axis=(1,2))  # Shape becomes (n, 1, 1)
    pdf = (np.exp(-(np.log(bins) - mean)**2 /
                  (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi)))
    return pdf 


In [ ]:
def N10_from_modes(Nmode, Dmode, nbins=100):
    '''
    For lognormal modes with number concentrations Nmode and diameters Dmode [m],
    evaluates modes over bins and then integrates for N10. N10 has same units as Nmode.
    '''
    # carry out some checks on dimensions
    if len(Nmode) != len(Dmode):
        print(f'[N10_from_modes] [WARNING] length of Nmode ({len(Nmode)}) doesnt match length of Dmode ({len(Dmode)})')
        print(f'[N10_from_modes] [WARNING] expect code to fail or produce unexpected behaviour')
    nmodes = len(Nmode)
    modes = [key for key in const.mode_sig]
    if len(Nmode) != len(modes):
        print(f'[N10_from_modes] [WARNING] length of Nmode ({len(Nmode)}) doesnt match length of modes ({len(modes)})')
        print(f'[N10_from_modes] [WARNING] expect code to fail or produce unexpected behaviour')
    dims = Nmode[0].shape
    bins = np.logspace(-9,-6, num=nbins)
    logD = np.log(bins)
    i_10nm = np.nonzero(bins < 10e-9)[0][-1] + 1
    dNdlogD = np.zeros((nbins,*dims))
    # loop over modes to sum up dN/dlogD
    for m,mode in enumerate(modes):
        D = Dmode[m].values
        N = Nmode[m].values
        sigma = const.mode_sig[mode]*np.ones(N.shape)
        pdf = lognormal_pdf(np.log(D*1e09), 
                            np.log(sigma),
                            bins*1e09)
        dist = pdf*N.data
        dNdlogD += dist
    N10 = np.trapz(dNdlogD[i_10nm:], x=logD[i_10nm:], axis=0)
    return N10


## Load aerosol outputs

In [ ]:
# load air density in kg m-3, to convert aerosol mixing ratios to concentrations
ocean_rho = cfn.load_air_density('u-cu657')
ocean_particle_density_of_air = ((ocean_rho.air_density/const.molar_mass_air) * const.avogadro_number)
coast_rho = cfn.load_air_density('u-cr700')
coast_particle_density_of_air = ((coast_rho.air_density/const.molar_mass_air) * const.avogadro_number)

modes = ['soluble_nucleation','soluble_aitken','soluble_accumulation',
         'soluble_coarse','insoluble_aitken']
mixing_ratio_stash = [
    # UM stash codes for aerosol number mixing ratios
    'm01s34i101',
    'm01s34i103',
    'm01s34i107',
    'm01s34i113',
    'm01s34i119'
]
diameter_stash = [
    # UM stash codes for aerosol mode diameters
    'm01s38i401',
    'm01s38i402',
    'm01s38i403',
    'm01s38i404',
    'm01s38i405'
]
# check if aerosol data output is saved
for stash in mixing_ratio_stash:
    cfn.check_file_exists('u-cr700', stash, verbose=True)
    cfn.check_file_exists('u-cu657', stash, verbose=True)
for stash in diameter_stash:
    cfn.check_file_exists('u-cr700', stash, verbose=True)
    cfn.check_file_exists('u-cu657', stash, verbose=True)

# load mixing ratios and convert to concentration, load diameters
ocean_mix_ratio = [xr.open_dataset(f'data/model/u-cu657_{stash}_CAP.nc')[f'STASH_{stash}'] for stash in mixing_ratio_stash]
ocean_aero_conc = [nmr*ocean_particle_density_of_air for nmr in ocean_mix_ratio]
ocean_diameters = [xr.open_dataset(f'data/model/u-cu657_{stash}_CAP.nc')[f'STASH_{stash}'] for stash in diameter_stash]
coast_mix_ratio = [xr.open_dataset(f'data/model/u-cr700_{stash}_CAP.nc')[f'STASH_{stash}'] for stash in mixing_ratio_stash]
coast_aero_conc = [nmr*coast_particle_density_of_air for nmr in coast_mix_ratio]
coast_diameters = [xr.open_dataset(f'data/model/u-cr700_{stash}_CAP.nc')[f'STASH_{stash}'] for stash in diameter_stash]


### Load output for ERA5_INITIAL

In [ ]:
era5_initial_code = 'u-cw894'
era5_initial_rho = cfn.load_air_density(era5_initial_code)
era5_initial_particle_density_of_air = ((era5_initial_rho.air_density/const.molar_mass_air) * const.avogadro_number)
for stash in mixing_ratio_stash:
    cfn.check_file_exists(era5_initial_code, stash, verbose=True)
for stash in diameter_stash:
    cfn.check_file_exists(era5_initial_code, stash, verbose=True)
era5_initial_mix_ratio = [xr.open_dataset(f'data/model/{era5_initial_code}_{stash}_CAP.nc')[f'STASH_{stash}'] for stash in mixing_ratio_stash]
era5_initial_aero_conc = [nmr*era5_initial_particle_density_of_air for nmr in era5_initial_mix_ratio]
era5_initial_diameters = [xr.open_dataset(f'data/model/{era5_initial_code}_{stash}_CAP.nc')[f'STASH_{stash}'] for stash in diameter_stash]


## Calculate N10

In [ ]:
ocean_ntime = ocean_aero_conc[0].shape
coast_ntime = coast_aero_conc[0].shape
ocean_N10_arr = N10_from_modes(ocean_aero_conc, ocean_diameters)
coast_N10_arr = N10_from_modes(coast_aero_conc, coast_diameters)

ocean_N10 = xr.DataArray(
    data=ocean_N10_arr,
    coords=ocean_aero_conc[0].coords,
    name='n10'
)
coast_N10 = xr.DataArray(
    data=coast_N10_arr,
    coords=coast_aero_conc[0].coords,
    name='n10'
)


### Calculate N10 for ERA5_INITIAL

In [ ]:
era5_initial_ntime = era5_initial_aero_conc[0].shape
era5_initial_N10_arr = N10_from_modes(era5_initial_aero_conc, era5_initial_diameters)
era5_initial_N10 = xr.DataArray(
    data=era5_initial_N10_arr,
    coords=era5_initial_aero_conc[0].coords,
    name='n10'
)



## Save to file

In [ ]:
filename = 'data/model/u-cu657_N10_CAP.nc'
ocean_N10.to_netcdf(filename)
filename = 'data/model/u-cr700_N10_CAP.nc'
coast_N10.to_netcdf(filename)
